In [31]:
#import
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime

In [32]:
clim5shifted = pd.read_csv('clim5shifted.csv', encoding='utf-8')
clim5shifted = clim5shifted.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'month2', 'date2', 'latlon'])
clim28shifted = pd.read_csv('clim28shifted.csv', encoding='utf-8')
clim28shifted = clim28shifted.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'month2', 'date2', 'latlon'])

In [33]:
#load and format testset
df_test = pd.read_csv("results_test.csv", encoding="utf-8")
df_test['err'] = df_test['xgboost_tmp2m_pred'] - df_test['tmp2m']
df_test['lon2'] = df_test['lon']-360
df_test['latlon'] = df_test['lat'].astype(str) + "|" + df_test['lon'].astype(str)
df_test['start_date'] = pd.to_datetime(df_test['start_date'])
df_test['month'] = pd.DatetimeIndex(df_test['start_date']).month
df_test['month2'] = df_test['month'].astype(str).str.zfill(2)
df_test['year'] = pd.DatetimeIndex(df_test['start_date']).year
df_test['yearmonth'] = df_test['year'].astype(str) + '-' + df_test['month2'].astype(str)
df_test['date'] = pd.DatetimeIndex(df_test['start_date']).day
df_test['date2'] = df_test['date'].astype(str).str.zfill(2)
df_test['err'] = round(df_test['err'],3)
df_test['phys_err'] = round(df_test['phys_err'],3)

#Seasons
def season (x):
    if x in [12,1,2]:
        return 'winter'
    elif x in [3,4,5]:
        return 'spring'
    elif x in [6,7,8]:
        return 'summer'
    elif x in [9,10,11]:
        return 'fall'

#Locations
locations = pd.read_csv("locations.csv", encoding="utf-8")
locations.rename(columns={"Location": "latlon"}, inplace=True)
locations.sort_values(by=['latlon'], ascending=False)


#Map season and location and extremes
df_test['season'] = df_test['month'].apply(season)
df_test = pd.merge(df_test, locations, on='latlon')
df_test['heatwave'] = df_test['tmp2m'] >34
df_test['coldwave'] = df_test['tmp2m'] <-10

#Functions / Metrics
def mbe (x):
    return round(np.mean(x),3)

def mae (x):
    return round(np.mean(abs(x)), 3)

def sd (x):
    return round(np.std(x), 3)

df_test['mdll'] = df_test['month2'] + "|" + df_test['date2'] + "|" + df_test['latlon']
df_test

,key,start_date,lat,lon,xgboost_tmp2m_pred_zscore,tmp2m_daily_mean,tmp2m_daily_std,tmp2m_zscore,tmp2m,xgboost_tmp2m_pred,...,yearmonth,date,date2,season,State,Sub-Region,Region,heatwave,coldwave,mdll
0,2019-10-01|27.25|261.75,2019-10-01,27.25,261.75,1.028229,22.870967,2.045002,-0.014837,22.840626,24.973697,...,2019-10,1,01,fall,Texas,West South Central,South,False,False,10|01|27.25|261.75
1,2019-10-08|27.25|261.75,2019-10-08,27.25,261.75,1.093624,21.721882,1.985609,-1.286955,19.166492,23.893391,...,2019-10,8,08,fall,Texas,West South Central,South,False,False,10|08|27.25|261.75
2,2019-10-15|27.25|261.75,2019-10-15,27.25,261.75,1.043933,20.675154,2.121438,-1.040800,18.467161,22.889792,...,2019-10,15,15,fall,Texas,West South Central,South,False,False,10|15|27.25|261.75
3,2019-10-22|27.25|261.75,2019-10-22,27.25,261.75,0.892237,19.451975,2.304340,-1.896681,15.081376,21.507993,...,2019-10,22,22,fall,Texas,West South Central,South,False,False,10|22|27.25|261.75
4,2019-10-29|27.25|261.75,2019-10-29,27.25,261.75,0.883454,18.518180,2.384524,-1.176233,15.713424,20.624798,...,2019-10,29,29,fall,Texas,West South Central,South,False,False,10|29|27.25|261.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26393,2021-06-29|47.25|273.75,2021-06-29,47.25,273.75,-0.061508,15.384313,2.808936,0.734029,17.446154,15.211542,...,2021-06,29,29,summer,Michigan,East North Central,Midwest,False,False,06|29|47.25|273.75
26394,2020-07-07|47.25|273.75,2020-07-07,47.25,273.75,-0.408582,16.091479,2.732136,-0.339037,15.165182,14.975177,...,2020-07,7,07,summer,Michigan,East North Central,Midwest,False,False,07|07|47.25|273.75
26395,2020-07-14|47.25|273.75,2020-07-14,47.25,273.75,-0.268441,16.598514,2.503675,-0.350005,15.722215,15.926424,...,2020-07,14,14,summer,Michigan,East North Central,Midwest,False,False,07|14|47.25|273.75
26396,2020-07-21|47.25|273.75,2020-07-21,47.25,273.75,-0.357121,16.739847,2.226671,0.118659,17.004061,15.944656,...,2020-07,21,21,summer,Michigan,East North Central,Midwest,False,False,07|21|47.25|273.75


In [34]:
df_test = pd.merge(df_test, clim5shifted, on='mdll')
df_test = pd.merge(df_test, clim28shifted, on='mdll')
df_test

,key,start_date,lat,lon,xgboost_tmp2m_pred_zscore,tmp2m_daily_mean,tmp2m_daily_std,tmp2m_zscore,tmp2m,xgboost_tmp2m_pred,...,State,Sub-Region,Region,heatwave,coldwave,mdll,5y climatology,5y_climf,28y climatology,28y_climf
0,2019-10-01|27.25|261.75,2019-10-01,27.25,261.75,1.028229,22.870967,2.045002,-0.014837,22.840626,24.973697,...,Texas,West South Central,South,False,False,10|01|27.25|261.75,22.407943,19.308337,26.046652,22.870967
1,2019-10-08|27.25|261.75,2019-10-08,27.25,261.75,1.093624,21.721882,1.985609,-1.286955,19.166492,23.893391,...,Texas,West South Central,South,False,False,10|08|27.25|261.75,22.441165,17.739520,24.252977,21.721881
2,2019-10-15|27.25|261.75,2019-10-15,27.25,261.75,1.043933,20.675154,2.121438,-1.040800,18.467161,22.889792,...,Texas,West South Central,South,False,False,10|15|27.25|261.75,22.130111,17.699571,24.302400,20.675154
3,2019-10-22|27.25|261.75,2019-10-22,27.25,261.75,0.892237,19.451975,2.304340,-1.896681,15.081376,21.507993,...,Texas,West South Central,South,False,False,10|22|27.25|261.75,19.188970,17.046117,22.604475,19.451974
4,2019-10-29|27.25|261.75,2019-10-29,27.25,261.75,0.883454,18.518180,2.384524,-1.176233,15.713424,20.624798,...,Texas,West South Central,South,False,False,10|29|27.25|261.75,17.238747,16.060638,21.669365,18.518180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26393,2021-06-29|47.25|273.75,2021-06-29,47.25,273.75,-0.061508,15.384313,2.808936,0.734029,17.446154,15.211542,...,Michigan,East North Central,Midwest,False,False,06|29|47.25|273.75,12.946551,15.699352,12.652040,15.384313
26394,2020-07-07|47.25|273.75,2020-07-07,47.25,273.75,-0.408582,16.091479,2.732136,-0.339037,15.165182,14.975177,...,Michigan,East North Central,Midwest,False,False,07|07|47.25|273.75,14.359718,15.994225,13.627860,16.091478
26395,2020-07-14|47.25|273.75,2020-07-14,47.25,273.75,-0.268441,16.598514,2.503675,-0.350005,15.722215,15.926424,...,Michigan,East North Central,Midwest,False,False,07|14|47.25|273.75,15.126063,15.712688,15.112576,16.598514
26396,2020-07-21|47.25|273.75,2020-07-21,47.25,273.75,-0.357121,16.739847,2.226671,0.118659,17.004061,15.944656,...,Michigan,East North Central,Midwest,False,False,07|21|47.25|273.75,15.914344,15.567243,15.676359,16.739847


In [35]:
df_test[['key', 'tmp2m_daily_mean', '5y climatology', '5y_climf', '28y climatology', '28y_climf']]

,key,tmp2m_daily_mean,5y climatology,5y_climf,28y climatology,28y_climf
0,2019-10-01|27.25|261.75,22.870967,22.407943,19.308337,26.046652,22.870967
1,2019-10-08|27.25|261.75,21.721882,22.441165,17.739520,24.252977,21.721881
2,2019-10-15|27.25|261.75,20.675154,22.130111,17.699571,24.302400,20.675154
3,2019-10-22|27.25|261.75,19.451975,19.188970,17.046117,22.604475,19.451974
4,2019-10-29|27.25|261.75,18.518180,17.238747,16.060638,21.669365,18.518180
...,...,...,...,...,...,...
26393,2021-06-29|47.25|273.75,15.384313,12.946551,15.699352,12.652040,15.384313
26394,2020-07-07|47.25|273.75,16.091479,14.359718,15.994225,13.627860,16.091478
26395,2020-07-14|47.25|273.75,16.598514,15.126063,15.712688,15.112576,16.598514
26396,2020-07-21|47.25|273.75,16.739847,15.914344,15.567243,15.676359,16.739847


In [36]:
df_test['clim_err_5'] = df_test['5y_climf'] - df_test['tmp2m']
df_test['clim_err_28'] = df_test['28y_climf'] - df_test['tmp2m']
df_test

,key,start_date,lat,lon,xgboost_tmp2m_pred_zscore,tmp2m_daily_mean,tmp2m_daily_std,tmp2m_zscore,tmp2m,xgboost_tmp2m_pred,...,Region,heatwave,coldwave,mdll,5y climatology,5y_climf,28y climatology,28y_climf,clim_err_5,clim_err_28
0,2019-10-01|27.25|261.75,2019-10-01,27.25,261.75,1.028229,22.870967,2.045002,-0.014837,22.840626,24.973697,...,South,False,False,10|01|27.25|261.75,22.407943,19.308337,26.046652,22.870967,-3.532288,0.030341
1,2019-10-08|27.25|261.75,2019-10-08,27.25,261.75,1.093624,21.721882,1.985609,-1.286955,19.166492,23.893391,...,South,False,False,10|08|27.25|261.75,22.441165,17.739520,24.252977,21.721881,-1.426973,2.555389
2,2019-10-15|27.25|261.75,2019-10-15,27.25,261.75,1.043933,20.675154,2.121438,-1.040800,18.467161,22.889792,...,South,False,False,10|15|27.25|261.75,22.130111,17.699571,24.302400,20.675154,-0.767591,2.207993
3,2019-10-22|27.25|261.75,2019-10-22,27.25,261.75,0.892237,19.451975,2.304340,-1.896681,15.081376,21.507993,...,South,False,False,10|22|27.25|261.75,19.188970,17.046117,22.604475,19.451974,1.964741,4.370598
4,2019-10-29|27.25|261.75,2019-10-29,27.25,261.75,0.883454,18.518180,2.384524,-1.176233,15.713424,20.624798,...,South,False,False,10|29|27.25|261.75,17.238747,16.060638,21.669365,18.518180,0.347214,2.804757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26393,2021-06-29|47.25|273.75,2021-06-29,47.25,273.75,-0.061508,15.384313,2.808936,0.734029,17.446154,15.211542,...,Midwest,False,False,06|29|47.25|273.75,12.946551,15.699352,12.652040,15.384313,-1.746801,-2.061841
26394,2020-07-07|47.25|273.75,2020-07-07,47.25,273.75,-0.408582,16.091479,2.732136,-0.339037,15.165182,14.975177,...,Midwest,False,False,07|07|47.25|273.75,14.359718,15.994225,13.627860,16.091478,0.829043,0.926296
26395,2020-07-14|47.25|273.75,2020-07-14,47.25,273.75,-0.268441,16.598514,2.503675,-0.350005,15.722215,15.926424,...,Midwest,False,False,07|14|47.25|273.75,15.126063,15.712688,15.112576,16.598514,-0.009527,0.876299
26396,2020-07-21|47.25|273.75,2020-07-21,47.25,273.75,-0.357121,16.739847,2.226671,0.118659,17.004061,15.944656,...,Midwest,False,False,07|21|47.25|273.75,15.914344,15.567243,15.676359,16.739847,-1.436818,-0.264214


In [37]:
#Functions / Metrics
def mbe (x):
    return round(np.mean(x),3)

def mae (x):
    return round(np.mean(abs(x)), 3)

def sd (x):
    return round(np.std(x), 3)

In [49]:
#Climatology comparisons by State
indexlist = ['State']

vayuhpivot = pd.pivot_table(data=df_test, index=indexlist, values='err', aggfunc=[mae])
vayuhpivot.columns = list(map("_".join, vayuhpivot.columns))
vayuhpivot.rename(columns={"mae_err":"Vayuh MAE"}, inplace=True)
vayuhpivot

clim5pivot = pd.pivot_table(data=df_test, index=indexlist, values='clim_err_5', aggfunc=[mae])
clim5pivot.columns = list(map("_".join, clim5pivot.columns))
clim5pivot.rename(columns={"mae_clim_err_5":"5y Clim MAE"}, inplace=True)
clim5pivot

clim28pivot = pd.pivot_table(data=df_test, index=indexlist, values='clim_err_28', aggfunc=[mae])
clim28pivot.columns = list(map("_".join, clim28pivot.columns))
clim28pivot.rename(columns={"mae_clim_err_28":"28y Clim MAE"}, inplace=True)
clim28pivot

chartdata = pd.merge(vayuhpivot, clim5pivot, on=indexlist)
chartdata = pd.merge(chartdata, clim28pivot, on=indexlist)
chartdata['Vayuh > 5y'] = chartdata['Vayuh MAE'] < chartdata['5y Clim MAE']
chartdata['Vayuh > 28y'] = chartdata['Vayuh MAE'] < chartdata['28y Clim MAE']
chartdata['MAE Reduction 5y'] = round((chartdata['5y Clim MAE'] - chartdata['Vayuh MAE'])/chartdata['5y Clim MAE'],3)
chartdata['MAE Reduction 28y'] = round((chartdata['28y Clim MAE'] - chartdata['Vayuh MAE'])/chartdata['28y Clim MAE'],3)
print('5y MAE Reduction: ' + 'Max: ' + str(chartdata['MAE Reduction 5y'].max()) + ' | ' + 'Min: ' + str(chartdata['MAE Reduction 5y'].min()))
print('28y MAE Reduction: ' + 'Max: ' + str(chartdata['MAE Reduction 28y'].max()) + ' | ' + 'Min: ' + str(chartdata['MAE Reduction 28y'].min()))
chartdata

5y MAE Reduction: Max: 0.518 | Min: 0.235
28y MAE Reduction: Max: 0.054 | Min: -0.225


,Vayuh MAE,5y Clim MAE,28y Clim MAE,Vayuh > 5y,Vayuh > 28y,MAE Reduction 5y,MAE Reduction 28y
State,,,,,,,
Alabama,1.708,2.484,1.575,True,False,0.312,-0.084
Arizona,1.606,3.334,1.697,True,True,0.518,0.054
Arkansas,1.731,3.054,1.563,True,False,0.433,-0.107
California,1.558,2.883,1.591,True,True,0.460,0.021
Colorado,1.812,3.366,1.764,True,False,0.462,-0.027
Florida,1.312,1.715,1.230,True,False,0.235,-0.067
Georgia,1.682,2.444,1.517,True,False,0.312,-0.109
Idaho,1.931,3.760,1.797,True,False,0.486,-0.075
Illinois,2.120,3.678,1.857,True,False,0.424,-0.142


In [50]:
#Climatology comparisons by season
indexlist = ['season']

vayuhpivot = pd.pivot_table(data=df_test, index=indexlist, values='err', aggfunc=[mae])
vayuhpivot.columns = list(map("_".join, vayuhpivot.columns))
vayuhpivot.rename(columns={"mae_err":"Vayuh MAE"}, inplace=True)
vayuhpivot

clim5pivot = pd.pivot_table(data=df_test, index=indexlist, values='clim_err_5', aggfunc=[mae])
clim5pivot.columns = list(map("_".join, clim5pivot.columns))
clim5pivot.rename(columns={"mae_clim_err_5":"5y Clim MAE"}, inplace=True)
clim5pivot

clim28pivot = pd.pivot_table(data=df_test, index=indexlist, values='clim_err_28', aggfunc=[mae])
clim28pivot.columns = list(map("_".join, clim28pivot.columns))
clim28pivot.rename(columns={"mae_clim_err_28":"28y Clim MAE"}, inplace=True)
clim28pivot

chartdata = pd.merge(vayuhpivot, clim5pivot, on=indexlist)
chartdata = pd.merge(chartdata, clim28pivot, on=indexlist)
chartdata['Vayuh > 5y'] = chartdata['Vayuh MAE'] < chartdata['5y Clim MAE']
chartdata['Vayuh > 28y'] = chartdata['Vayuh MAE'] < chartdata['28y Clim MAE']
chartdata['MAE Reduction 5y'] = round((chartdata['5y Clim MAE'] - chartdata['Vayuh MAE'])/chartdata['5y Clim MAE'],3)
chartdata['MAE Reduction 28y'] = round((chartdata['28y Clim MAE'] - chartdata['Vayuh MAE'])/chartdata['28y Clim MAE'],3)
print('5y MAE Reduction: ' + 'Max: ' + str(chartdata['MAE Reduction 5y'].max()) + ' | ' + 'Min: ' + str(chartdata['MAE Reduction 5y'].min()))
print('28y MAE Reduction: ' + 'Max: ' + str(chartdata['MAE Reduction 28y'].max()) + ' | ' + 'Min: ' + str(chartdata['MAE Reduction 28y'].min()))
chartdata

5y MAE Reduction: Max: 0.538 | Min: 0.286
28y MAE Reduction: Max: 0.026 | Min: -0.193


,Vayuh MAE,5y Clim MAE,28y Clim MAE,Vayuh > 5y,Vayuh > 28y,MAE Reduction 5y,MAE Reduction 28y
season,,,,,,,
fall,2.445,3.422,2.050,True,False,0.286,-0.193
spring,1.726,3.735,1.610,True,False,0.538,-0.072
summer,1.260,2.229,1.294,True,True,0.435,0.026
winter,2.647,4.231,2.569,True,False,0.374,-0.030


In [51]:
#Climatology comparisons by yearmonth
indexlist = ['yearmonth']

vayuhpivot = pd.pivot_table(data=df_test, index=indexlist, values='err', aggfunc=[mae])
vayuhpivot.columns = list(map("_".join, vayuhpivot.columns))
vayuhpivot.rename(columns={"mae_err":"Vayuh MAE"}, inplace=True)
vayuhpivot

clim5pivot = pd.pivot_table(data=df_test, index=indexlist, values='clim_err_5', aggfunc=[mae])
clim5pivot.columns = list(map("_".join, clim5pivot.columns))
clim5pivot.rename(columns={"mae_clim_err_5":"5y Clim MAE"}, inplace=True)
clim5pivot

clim28pivot = pd.pivot_table(data=df_test, index=indexlist, values='clim_err_28', aggfunc=[mae])
clim28pivot.columns = list(map("_".join, clim28pivot.columns))
clim28pivot.rename(columns={"mae_clim_err_28":"28y Clim MAE"}, inplace=True)
clim28pivot

chartdata = pd.merge(vayuhpivot, clim5pivot, on=indexlist)
chartdata = pd.merge(chartdata, clim28pivot, on=indexlist)
chartdata['Vayuh > 5y'] = chartdata['Vayuh MAE'] < chartdata['5y Clim MAE']
chartdata['Vayuh > 28y'] = chartdata['Vayuh MAE'] < chartdata['28y Clim MAE']
chartdata['MAE Reduction 5y'] = round((chartdata['5y Clim MAE'] - chartdata['Vayuh MAE'])/chartdata['5y Clim MAE'],3)
chartdata['MAE Reduction 28y'] = round((chartdata['28y Clim MAE'] - chartdata['Vayuh MAE'])/chartdata['28y Clim MAE'],3)
print('5y MAE Reduction: ' + 'Max: ' + str(chartdata['MAE Reduction 5y'].max()) + ' | ' + 'Min: ' + str(chartdata['MAE Reduction 5y'].min()))
print('28y MAE Reduction: ' + 'Max: ' + str(chartdata['MAE Reduction 28y'].max()) + ' | ' + 'Min: ' + str(chartdata['MAE Reduction 28y'].min()))
chartdata

5y MAE Reduction: Max: 0.67 | Min: -0.396
28y MAE Reduction: Max: 0.198 | Min: -0.708


,Vayuh MAE,5y Clim MAE,28y Clim MAE,Vayuh > 5y,Vayuh > 28y,MAE Reduction 5y,MAE Reduction 28y
yearmonth,,,,,,,
2019-01,3.243,4.609,2.859,True,False,0.296,-0.134
2019-02,3.300,6.642,3.182,True,False,0.503,-0.037
2019-03,1.121,3.316,0.929,True,False,0.662,-0.207
2019-04,1.652,4.470,1.705,True,True,0.630,0.031
2019-05,1.653,3.733,1.524,True,False,0.557,-0.085
2019-06,1.091,1.796,1.048,True,False,0.393,-0.041
2019-07,1.083,1.905,0.954,True,False,0.431,-0.135
2019-08,1.712,4.108,1.820,True,True,0.583,0.059
2019-09,2.648,3.745,2.477,True,False,0.293,-0.069
